<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/shaptonstall/brace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Mount google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/Github/capstone/brace

/content/drive/MyDrive/Github/capstone/brace


In [6]:
%ls

annotations/    brace_test.pth   brace_train.pth  dataset_pytorch.py  __pycache__/  utils/
brace_test.pkl  brace_train.pkl  dataset/         figures/            README.md     videos_info.csv


In [7]:
# !nvidia-smi

In [8]:
import sys
sys.path.append('/content/drive/MyDrive/Github/capstone/brace')

In [9]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
import dataset_pytorch
from pathlib import Path
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import TensorDataset
from tqdm import tqdm
from pathlib import PosixPath
import pickle

In [19]:
# Load dataset, only need to use once since models are saved

def prepare_data():
    # Load data and create datasets
    sequences_path_ = Path('/content/drive/MyDrive/Github/capstone/brace/dataset')
    df_ = pd.read_csv(Path('/content/drive/MyDrive/Github/capstone/brace/annotations/sequences.csv'))

    train_df = pd.read_csv('/content/drive/MyDrive/Github/capstone/brace/annotations/sequences_train.csv')
    train_df = df_[df_.uid.isin(train_df.uid)]

    brace_train = dataset_pytorch.BraceDataset(sequences_path_, train_df)
    print(f'Loaded BRACE training set! We got {len(brace_train)} training sequences')
    skeletons_train, metadata_train = brace_train.__getitem__(0)
    print(metadata_train)

    test_df = pd.read_csv('/content/drive/MyDrive/Github/capstone/brace/annotations/sequences_test.csv')
    test_df = df_[df_.uid.isin(test_df.uid)]

    brace_test = dataset_pytorch.BraceDataset(sequences_path_, test_df)
    print(f'Loaded BRACE test set! We got {len(brace_test)} testing sequences')
    skeletons_test, metadata_test = brace_test.__getitem__(0)
    print(metadata_test)

    return brace_train, brace_test, skeletons_train, metadata_train, skeletons_test, metadata_test

brace_train, brace_test, skeletons_train, metadata_train, skeletons_test, metadata_test = prepare_data()

# Continue with the rest of the code (define model, train, test)

Loading BRACE: 100%|██████████| 319/319 [00:20<00:00, 15.26it/s]


Going to sample 900 frames from each sequence. Average sequence length is 730.7711598746082
Loaded BRACE training set! We got 319 training sequences
{'video_id': '3rIk56dcBTM', 'seq_idx': 1, 'start_frame': 1751, 'end_frame': 2915, 'dancer': 'roxrite', 'dancer_id': 47, 'year': 2011, 'uid': '3rIk56dcBTM.1'}


Loading BRACE: 100%|██████████| 146/146 [00:09<00:00, 14.83it/s]

Going to sample 900 frames from each sequence. Average sequence length is 690.9794520547945
Loaded BRACE test set! We got 146 testing sequences
{'video_id': '3rIk56dcBTM', 'seq_idx': 0, 'start_frame': 851, 'end_frame': 1645, 'dancer': 'el niño', 'dancer_id': 10, 'year': 2011, 'uid': '3rIk56dcBTM.0'}


In [20]:
# # Save the entire dataset object
# with open('brace_train.pkl', 'wb') as f:
#     pickle.dump(brace_train, f)

# with open('brace_test.pkl', 'wb') as f:
#     pickle.dump(brace_test, f)

Use this to load the dataset

In [ ]:
# Load the entire dataset object
with open('brace_train.pkl', 'rb') as f:
    brace_train = pickle.load(f)

with open('brace_test.pkl', 'rb') as f:
    brace_test = pickle.load(f)

In [11]:
x_train= brace_train.sequences
y_train=brace_train.clip_labels
x_test = brace_test.sequences
y_test=brace_test.clip_labels
len(x_train), len(y_train), len(x_test), len(y_test)

(319, 319, 146, 146)

In [12]:
# Convert NumPy arrays to PyTorch tensors
x_train_tensors = [torch.tensor(seq) for seq in x_train]
x_test_tensors = [torch.tensor(seq) for seq in x_test]

# Pad sequences to a fixed length
padded_x_train = rnn_utils.pad_sequence(x_train_tensors, batch_first=True)
padded_x_test = rnn_utils.pad_sequence(x_test_tensors, batch_first=True)

# Convert to NumPy arrays
y_train_array = np.array(y_train)
y_test_array = np.array(y_test)

# Convert to PyTorch tensors
y_train_tensor = torch.tensor(y_train_array)
y_test_tensor = torch.tensor(y_test_array)

# Create datasets and data loaders using padded sequences
train_dataset = TensorDataset(padded_x_train, y_train_tensor)
test_dataset = TensorDataset(padded_x_test, y_test_tensor)

# Define batch size
batch_size = 32

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=None, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=None, shuffle=False)

In [13]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [14]:
# Step 3: Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
# Assuming you have a DataLoader named train_loader

# Instantiate your LSTMModel
input_size = 2  # adjust based on your input data
hidden_size = 128
num_layers = 2
num_classes = 3  # adjust based on your dataset
lstm_model = LSTMModel(input_size, hidden_size, num_layers, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    lstm_model.train()  # Set the model to training mode
    running_loss = 0.0

    # Iterate over the data
    for batch_x, batch_y in train_loader:
        # Forward pass
        batch_x = batch_x.float()
        batch_y = torch.full((batch_x.size(0),), batch_y.item(), dtype=torch.long)

        outputs = lstm_model(batch_x)
        # print(batch_y.shape, outputs.shape)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_x.size(0)

    # Calculate average loss for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)

    # Print loss (optional)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

print('Finished Training')

Epoch [1/10], Loss: 1588.8892
Epoch [2/10], Loss: 1520.1132
Epoch [3/10], Loss: 1521.6549
Epoch [4/10], Loss: 1520.8709
Epoch [5/10], Loss: 1519.7987
Epoch [6/10], Loss: 1518.9312
Epoch [7/10], Loss: 1517.8251
Epoch [8/10], Loss: 1516.7940
Epoch [9/10], Loss: 1515.6855
Epoch [10/10], Loss: 1514.5620
Finished Training


In [21]:
# Define the file path where you want to save the model
model_path = 'lstm_model.pth'

# # Save the model state dictionary
# torch.save(lstm_model.state_dict(), model_path)

# print("Model saved successfully.")

Model saved successfully.


In [22]:
# Load the model
loaded_model = LSTMModel(input_size, hidden_size, num_layers, num_classes)

# Load the model state dictionary
loaded_model.load_state_dict(torch.load(model_path))

print("Model loaded successfully.")

Model loaded successfully.


In [23]:
# Assuming you have a DataLoader named test_loader

# Set the model to evaluation mode
lstm_model.eval()

# Initialize variables to track accuracy and loss
total_correct = 0
total_samples = 0
total_loss = 0.0

# Disable gradient calculation during testing
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        # Forward pass
        batch_x = batch_x.float()
        batch_y = torch.full((batch_x.size(0),), batch_y.item(), dtype=torch.long)

        outputs = lstm_model(batch_x)
        loss = criterion(outputs, batch_y)

        # Calculate total loss
        total_loss += loss.item() * batch_x.size(0)

        # Get predictions
        _, predicted = torch.max(outputs, 1)

        # Update total number of samples and correct predictions
        total_correct += (predicted == batch_y).sum().item()
        total_samples += batch_x.size(0)

# Calculate average loss and accuracy
average_loss = total_loss / total_samples
accuracy = total_correct / total_samples

# Print test results
print(f'Test Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}')

Test Loss: 0.7476, Accuracy: 0.5959
